In [25]:
import gym
from gym import Env
from gym.spaces import Discrete,Box,Dict,Tuple,MultiBinary,MultiDiscrete


import numpy as np
import random
import os


from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [26]:
# Types of Spaces 

Discrete(3).sample()

1

In [27]:
Box(0,1,shape=(3,)).sample()

array([0.8118124 , 0.11793407, 0.26997036], dtype=float32)

In [28]:
Tuple((Discrete(3),Box(0,1,shape=(2,))))

Tuple(Discrete(3), Box(0.0, 1.0, (2,), float32))

In [29]:
Dict({'height':Discrete(2),'speed':Box(0,100,shape=(1,)),'color':MultiBinary(3),"something":MultiDiscrete([3,2,2,2])}).sample()

OrderedDict([('color', array([0, 1, 1], dtype=int8)),
             ('height', 1),
             ('something', array([1, 1, 1, 0], dtype=int64)),
             ('speed', array([54.13773], dtype=float32))])

In [30]:
# Build an agent to give us the best shower possible 
# Randomly temperature 
# 37 and 39 degrees

In [31]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)# have 3 valuse to be controlled
        self.observation_space = Box(low=0,high=100,shape=(1,))
        self.state = 38+ random.randint(-3,3)
        self.shower_length = 60
        

    def step(self, action):
        #Apply temp adj
        self.state += action-1

        # Decrease shower time
        self.shower_length -=1

        #Calculate reward
        if self.state >= 37 and self.state <= 39:
            reward = 1
        else:
            reward = -10
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}

        return self.state,reward,done,info
    def render(self):
        pass
    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state
        

In [32]:
env = ShowerEnv()
env.reset()

array([40.])

In [33]:
env.observation_space.sample()

array([55.25589], dtype=float32)

In [34]:
env.action_space.sample()

2

In [35]:

episodes = 7
for episode in range(1,episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = env.action_space.sample()
        n_states,reward,done,info = env.step(action)
        score+=reward
    print(f"Episode:({episode}) Score:({score})")
env.close()

Episode:(1) Score:(0)
Episode:(2) Score:(0)
Episode:(3) Score:(0)
Episode:(4) Score:(0)
Episode:(5) Score:(0)
Episode:(6) Score:(0)


In [36]:
log_path = os.path.join('Training','Logs')
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [37]:
model.learn(total_timesteps=100000)

Logging to Training\Logs\PPO_22
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -416     |
| time/              |          |
|    fps             | 642      |
|    iterations      | 1        |
|    time_elapsed    | 3        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 60         |
|    ep_rew_mean          | -436       |
| time/                   |            |
|    fps                  | 551        |
|    iterations           | 2          |
|    time_elapsed         | 7          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01269181 |
|    clip_fraction        | 0.066      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.09      |
|    explained_variance   | 2.01e-05   |
|    learning

In [38]:
model.save(os.path.join("Training",'Saved Models','Shower Model'))

In [39]:
evaluate_policy(model,env,n_eval_episodes=10)

c:\Users\arya2\AppData\Local\Programs\Python\Python311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-336.0, 323.3326460473795)

In [49]:
env = ShowerEnv()
episodes = 7
for episode in range(1,episodes):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        action,_ = model.predict(obs)
        obs,reward,done,info = env.step(action)
        score+=reward
    print(f"Episode:({episode}) Score:({score})")
env.close()

Episode:(1) Score:(-314)
Episode:(2) Score:(-369)
Episode:(3) Score:(60)
Episode:(4) Score:(-259)
Episode:(5) Score:(-446)
Episode:(6) Score:(-600)
